<a href="https://colab.research.google.com/github/jsuj1th/Colab/blob/main/DL_project1_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tamu_csce_636_project1

In [ ]:
# Dataset_generation:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


from mpi4py import MPI
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from itertools import combinations, product
import os
from tqdm import tqdm

# MPI setup
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Set a unique random seed per job and rank to ensure different matrix generation
np.random.seed(3)

output_dir = "/scratch/user/haikookhandor/DL/datasets_mpi/"
os.makedirs(output_dir, exist_ok=True)

# Parameters
max_valid_samples = 10000 # total valid samples per rank for debugging
save_threshold_bytes = 5 * 1024 * 1024

# (n, k, m) combinations
allowed_combinations = [
    (9, 4, [2, 3,4,5]),
    (9, 5, [2, 3,4]),
    (9, 6, [2,3]),
    (10, 4, [2, 3,4,5,6]),
    (10, 5, [2, 3,4,5]),
    (10, 6, [2,3,4]),
]
triplets = [(n, k, m) for n, k, ms in allowed_combinations for m in ms]

# Divide combinations across ranks
combo_chunks = np.array_split(triplets, size)
my_combos = combo_chunks[rank]

def generate_all_tuples(n, m):
    for a in range(n):
        for b in range(n):
            if b == a:
                continue
            rest = [i for i in range(n) if i not in {a, b}]
            for X in combinations(rest, m - 1):
                for psi in product([-1, 1], repeat=m):
                    yield (a, b, X, psi)

def solve_lp(G, k, n, m, a, b, X, psi):
    Y = [i for i in range(n) if i not in set(X).union({a, b})]
    x_list = [a] + sorted(X) + [b] + sorted(Y)
    tau_inv = {v: i for i, v in enumerate(x_list)}
    c = -psi[0] * G[:, a]
    A_ub, b_ub = [], []

    for j in X:
        row = (psi[tau_inv[j]] * G[:, j] - psi[0] * G[:, a])
        A_ub.append(row)
        b_ub.append(0)
        row = -psi[tau_inv[j]] * G[:, j]
        A_ub.append(row)
        b_ub.append(-1)

    for j in Y:
        A_ub.append(G[:, j])
        b_ub.append(1)
        A_ub.append(-G[:, j])
        b_ub.append(1)

    A_eq = [G[:, b]]
    b_eq = [1]

    res = linprog(c, A_ub=np.array(A_ub), b_ub=np.array(b_ub),
                  A_eq=np.array(A_eq), b_eq=np.array(b_eq),
                  method='highs')

    if res.status == 0:
        return -res.fun
    elif res.status == 3:
        return np.inf
    else:
        return 0

def compute_m_height(G, k, n, m):
    max_h = 1
    for a, b, X, psi in generate_all_tuples(n, m):
        h = solve_lp(G, k, n, m, a, b, X, psi)
        if h == np.inf:
            return np.inf
        if h > max_h:
            max_h = h
    return max_h

# Initialize global sample buffer
samples = []
file_counter = 0
total_valid_samples = 0

with tqdm(total=max_valid_samples, disable=(rank != 0)) as pbar:
    while total_valid_samples < max_valid_samples:
        for n, k, m in my_combos:
            if total_valid_samples >= max_valid_samples:
                break

            P = np.random.uniform(-100, 100, size=(k, n - k))
            G = np.hstack([np.eye(k), P])
            h_m = compute_m_height(G, k, n, m)

            if h_m != np.inf and h_m > 0:
                samples.append({
                    "n": n,
                    "k": k,
                    "m": m,
                    "h_m": h_m,
                    "P": P.flatten().tolist()
                })
                total_valid_samples += 1
                pbar.update(1)

                if len(samples) >= 10:
                    df = pd.DataFrame(samples)
                    mem_size = df.memory_usage(deep=True).sum()
                    if mem_size >= save_threshold_bytes:
                        filename = f"{output_dir}/rank{rank}_part{file_counter}.csv"
                        df.to_csv(filename, index=False)
                        print(f"[Rank {rank}] Saved {filename} ({mem_size / (1024**2):.2f} MB)")
                        file_counter += 1
                        samples = []

# Save any remaining samples
if samples:
    df = pd.DataFrame(samples)
    filename = f"{output_dir}/rank{rank}_final.csv"
    df.to_csv(filename, index=False)
    print(f"[Rank {rank}] Saved {filename} (final)")

In [19]:
# Import required libraries
from tamu_csce_636_project1 import Evaluator
import torch
import torch.nn as nn
import numpy as np
import joblib

# Initialize the evaluator
evaluator = Evaluator(
    first_name="Sujith",
    last_name="Julakantu",
    email="s02@tamu.edu",
    print=False,
)

# Define paths and constants
MODEL_SAVE_DIR = 'saved_models'
MODEL_PATH = f'{MODEL_SAVE_DIR}/best_model_1.pt'
SCALER_PATH = f'{MODEL_SAVE_DIR}/scaler.pkl'
EXPECTED_P_LEN = 25

class MHeight(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
             nn.Linear(input_dim, 512), nn.BatchNorm1d(512),nn.ReLU(),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.Tanh(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.BatchNorm1d(32), nn.Tanh(),
            nn.Linear(32, 16), nn.BatchNorm1d(16), nn.ReLU(),
            nn.Linear(16, 1), nn.Softplus()
        )

    def forward(self, x):
        return self.net(x) + 1

def preprocess_for_evaluator(n, k, m, p_list, max_nk=EXPECTED_P_LEN, scaler=None):
    """
    Preprocess a list of (n, k, m, P) inputs for the model.

    Args:
        n (int): Number of columns in G
        k (int): Number of rows in G
        m (int): m-height index
        p_list (list): List of P matrices (numpy arrays)
        max_nk (int): Maximum length for padding
        scaler (StandardScaler): Loaded scaler object

    Returns:
        torch.Tensor: Preprocessed batch tensor
    """
    # padding as added for maintaining uniform input size
    X_batch = []
    for P in p_list:
        P_array = np.array(P, dtype=np.float32).flatten()
        Padded_G = np.zeros(max_nk, dtype=np.float32)
        Padded_G[:len(P_array)] = P_array
        X = np.concatenate([np.array([n, k, m], dtype=np.float32), Padded_G])
        X_batch.append(X)

    X_batch = np.array(X_batch)
    X_scaled = scaler.transform(X_batch)
    return torch.tensor(X_scaled, dtype=torch.float32)


def predict_m_height(n, k, m, p_list):
    """
    Parameters:
        n (int)
        k (int)
        m (int)
        p_list (list): List of P matrices (numpy arrays)

    Returns:
        list: Predicted m-height values
    """
    # Load model and scaler
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_dim = 3 + EXPECTED_P_LEN
    model = MHeight(input_dim).to(device)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.eval()
    scaler = joblib.load(SCALER_PATH)

    # Preprocess input
    input_tensor = preprocess_for_evaluator(n, k, m, p_list, EXPECTED_P_LEN, scaler).to(device)

    # Make predictions
    with torch.no_grad():
        predictions = model(input_tensor).cpu().numpy().flatten()

    return predictions.tolist()

# Evaluate with the provided example
σ = evaluator.eval(
    inputs={
        '[5,2,2]': [
            np.array([
                [0.4759809, 0.9938236, 0.819425],
                [-0.8960798, -0.7442706, 0.3345122],
            ]),
        ],
    },
    outputs={
        '[5,2,2]': [1.9242387],
    },
    func=predict_m_height,
)

# Print evaluation result
print("Evaluation result:", dict(σ))

Evaluation result: {(5, 2, 2): 61.362821524501605}
